# Dataset: Banco de Imagens

##Objetivo
- Classificação de Imagens utilizando PyTorch e Yolov5

##Autor

#####Enio Kilder Oliveira da Silva            
Turma: 2023.1

## Preparação

Carregando as Bibliotecas

In [1]:
%matplotlib inline
import matplotlib.pyplot as plot
import os
from IPython.display import display
import numpy as np
import pandas as pd
import tensorflow as tf
#from huggingface_hub import push_to_hub_keras
import scipy.stats as stats
import sklearn
import statsmodels.api as sm
import seaborn as sns

In [2]:
!git clone https://github.com/ultralytics/yolov5  # clone
%cd yolov5
%pip install -qr requirements.txt  # install

import torch
import utils
display = utils.notebook_init()  # checks

YOLOv5 🚀 v7.0-230-g53efd07 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)


Setup complete ✅ (2 CPUs, 12.7 GB RAM, 27.1/78.2 GB disk)


Preparando o ambiente

In [3]:
!pip install roboflow

from roboflow import Roboflow
rf = Roboflow(api_key="YCjh46qKuqpCtCnbreiF")
project = rf.workspace("eniokilder").project("banco-imagem")
dataset = project.version(1).download("folder")

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 1.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 6.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 178.7/178.7 kB 21.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.8/58.8 kB 8.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 MB 12.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.8/67.8 kB 8.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 72.2/72.2 kB 10.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 54.5/54.5 kB 7.6 MB/s eta 0:00:00
  Attempting uninstall: pyparsing
    Found existing installation: pyparsing 3.1.1
    Uninstalling pyparsing-3.1.1:
      Successfully uninstalled pyparsing-3.1.1
  Attempting uninstall: opencv-python-headless
    Found existing installation: opencv-python-headless 4.8.1.78
    Uninstalling opencv-python-headless-4.8.1.78:
      Successfully uninstal

loading Roboflow workspace...
loading Roboflow project...



Extracting Dataset Version Zip to Banco-Imagem-1 in folder:: 100%|██████████| 382/382 [00:00<00:00, 4349.38it/s]


Baixando os modelos treinados do ImageNet pré-treinados no ImageNet usando YOLOv5 Utils

In [4]:
%cd ../yolov5
from utils.downloads import attempt_download
#'n'=avioes, 's'=carros, 'm'=motos, 'l'=barcos, 'x'=helicopteros
p5 = ['n', 's', 'm', 'l', 'x']  # P5 models
cls = [f'{x}-cls' for x in p5]  # classification models

for x in cls:
    attempt_download(f'weights/yolov5{x}.pt')

/content/yolov5


100%|██████████| 4.87M/4.87M [00:00<00:00, 36.1MB/s]

100%|██████████| 10.5M/10.5M [00:00<00:00, 96.4MB/s]

100%|██████████| 24.9M/24.9M [00:00<00:00, 182MB/s]

100%|██████████| 50.9M/50.9M [00:00<00:00, 234MB/s]

100%|██████████| 92.0M/92.0M [00:00<00:00, 211MB/s]



In [5]:
import yaml
with open( "/content/yolov5/models/yolov5n.yaml", 'r') as stream:
    num_classes = str(yaml.safe_load(stream)['nc'])

In [6]:
%cat /content/yolov5/models/yolov5n.yaml

# YOLOv5 🚀 by Ultralytics, AGPL-3.0 license

# Parameters
nc: 80  # number of classes
depth_multiple: 0.33  # model depth multiple
width_multiple: 0.25  # layer channel multiple
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 v6.0 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Conv, [64, 6, 2, 2]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, C3, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 6, C3, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, C3, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 3, C3, [1024]],
   [-1, 1, SPPF, [1024, 5]],  # 9
  ]

# YOLOv5 v6.0 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, C3, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]

In [7]:
from IPython.core.magic import register_line_cell_magic

@register_line_cell_magic
def writetemplate(line, cell):
    with open(line, 'w') as f:
        f.write(cell.format(**globals()))

In [8]:
%%writetemplate /content/yolov5/models/custom_yolov5s.yaml

# parameters
nc: {num_classes}  # number of classes

depth_multiple: 0.33  # model depth multiple
width_multiple: 0.50  # layer channel multiple

# anchors
anchors:
  - [10,13, 16,30, 33,23]  # P3/8
  - [30,61, 62,45, 59,119]  # P4/16
  - [116,90, 156,198, 373,326]  # P5/32

# YOLOv5 backbone
backbone:
  # [from, number, module, args]
  [[-1, 1, Focus, [64, 3]],  # 0-P1/2
   [-1, 1, Conv, [128, 3, 2]],  # 1-P2/4
   [-1, 3, BottleneckCSP, [128]],
   [-1, 1, Conv, [256, 3, 2]],  # 3-P3/8
   [-1, 9, BottleneckCSP, [256]],
   [-1, 1, Conv, [512, 3, 2]],  # 5-P4/16
   [-1, 9, BottleneckCSP, [512]],
   [-1, 1, Conv, [1024, 3, 2]],  # 7-P5/32
   [-1, 1, SPP, [1024, [5, 9, 13]]],
   [-1, 3, BottleneckCSP, [1024, False]],  # 9
  ]

# YOLOv5 head
head:
  [[-1, 1, Conv, [512, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 6], 1, Concat, [1]],  # cat backbone P4
   [-1, 3, BottleneckCSP, [512, False]],  # 13

   [-1, 1, Conv, [256, 1, 1]],
   [-1, 1, nn.Upsample, [None, 2, 'nearest']],
   [[-1, 4], 1, Concat, [1]],  # cat backbone P3
   [-1, 3, BottleneckCSP, [256, False]],  # 17 (P3/8-small)

   [-1, 1, Conv, [256, 3, 2]],
   [[-1, 14], 1, Concat, [1]],  # cat head P4
   [-1, 3, BottleneckCSP, [512, False]],  # 20 (P4/16-medium)

   [-1, 1, Conv, [512, 3, 2]],
   [[-1, 10], 1, Concat, [1]],  # cat head P5
   [-1, 3, BottleneckCSP, [1024, False]],  # 23 (P5/32-large)

   [[17, 20, 23], 1, Detect, [nc, anchors]],  # Detect(P3, P4, P5)
  ]

## Treinando e Validando os Dados Personalizados

In [9]:
dataset_name = dataset.location.split(os.sep)[-1]
os.environ["DATASET_NAME"] = dataset_name

Treinando sobre um modelo personalizado

In [10]:
#'n'=avioes, 's'=carros, 'm'=motos, 'l'=barcos, 'x'=helicopteros
%%time
%cd ../yolov5
!python classify/train.py --model yolov5n-cls.pt --data $DATASET_NAME --epochs 128 --batch 16 --img 320 --pretrained weights/yolov5n-cls.pt

/content/yolov5
2023-10-28 01:49:35.242300: E tensorflow/compiler/xla/stream_executor/cuda/cuda_dnn.cc:9342] Unable to register cuDNN factory: Attempting to register factory for plugin cuDNN when one has already been registered
2023-10-28 01:49:35.242363: E tensorflow/compiler/xla/stream_executor/cuda/cuda_fft.cc:609] Unable to register cuFFT factory: Attempting to register factory for plugin cuFFT when one has already been registered
2023-10-28 01:49:35.242406: E tensorflow/compiler/xla/stream_executor/cuda/cuda_blas.cc:1518] Unable to register cuBLAS factory: Attempting to register factory for plugin cuBLAS when one has already been registered
classify/train: model=yolov5n-cls.pt, data=Banco-Imagem-1, epochs=128, batch_size=16, imgsz=320, nosave=False, cache=None, device=, workers=8, project=runs/train-cls, name=exp, exist_ok=False, pretrained=weights/yolov5n-cls.pt, optimizer=Adam, lr0=0.001, decay=5e-05, label_smoothing=0.1, cutoff=None, dropout=None, verbose=False, seed=0, local_r

## Validando o modelo personalizado

In [11]:
!python classify/val.py --weights runs/train-cls/exp/weights/best.pt --data $DATASET_NAME

classify/val: data=Banco-Imagem-1, weights=['runs/train-cls/exp/weights/best.pt'], batch_size=128, imgsz=224, device=, workers=8, verbose=True, project=runs/val-cls, name=exp, exist_ok=False, half=False, dnn=False
YOLOv5 🚀 v7.0-230-g53efd07 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 1214869 parameters, 0 gradients, 2.9 GFLOPs
testing: 100% 1/1 [00:00<00:00,  1.05it/s]
                   Class      Images    top1_acc    top5_acc
                     all          36       0.639           1
                  avioes           7       0.571           1
                  barcos           6       0.667           1
                  carros          11       0.545           1
            helicopteros           8       0.875           1
                   motos           4         0.5           1
Speed: 0.1ms pre-process, 14.8ms inference, 0.6ms post-process per image at shape (1, 3, 224, 224)
Results saved to runs/val-cls/exp


##Inferindo com o modelo personalizado

In [14]:
  #'1'=avioes, '4'=carros, '2'=motos, '3'=barcos, '0'=helicopteros

In [33]:
#Pega a localização de uma imagem do conjunto de testes ou validações
if os.path.exists(os.path.join(dataset.location, "test")):
  split_path = os.path.join(dataset.location, "test")
else:
  os.path.join(dataset.location, "valid")
example_class = os.listdir(split_path)[4]
example_image_name = os.listdir(os.path.join(split_path, example_class))[4]
example_image_path = os.path.join(split_path, example_class, example_image_name)
os.environ["TEST_IMAGE_PATH"] = example_image_path

print(f"Inferindo sobre um exemplo da classe '{example_class}'")

#Infer
!python classify/predict.py --weights runs/train-cls/exp/weights/best.pt --source $TEST_IMAGE_PATH

Inferindo sobre um exemplo da classe 'carros'
classify/predict: weights=['runs/train-cls/exp/weights/best.pt'], source=/content/yolov5/Banco-Imagem-1/test/carros/00012_jpg.rf.9f0d32646e83139878c5788b040038f7.jpg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-230-g53efd07 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 1214869 parameters, 0 gradients, 2.9 GFLOPs
image 1/1 /content/yolov5/Banco-Imagem-1/test/carros/00012_jpg.rf.9f0d32646e83139878c5788b040038f7.jpg: 224x224 carros 0.91, avioes 0.08, motos 0.01, helicopteros 0.00, barcos 0.00, 2.7ms
Speed: 0.3ms pre-process, 2.7ms inference, 5.1ms NMS per image at shape (1, 3, 224, 224)
Results saved to runs/predict-cls/exp14


##Inferindo vários exemplos de imagens do imgur

In [16]:
# Aviao
import requests
image_url = "https://i.imgur.com/bqs2TxC.jpg"
response = requests.get(image_url)
response.raise_for_status()
with open('aviao.jpg', 'wb') as handler:
    handler.write(response.content)

In [41]:
!python classify/predict.py --weights ./weights/yolov5n-cls.pt  --source aviao.jpg

classify/predict: weights=['./weights/yolov5n-cls.pt'], source=aviao.jpg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-230-g53efd07 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 2489464 parameters, 0 gradients, 3.9 GFLOPs
image 1/1 /content/yolov5/aviao.jpg: 224x224 airliner 0.79, military aircraft 0.11, wing 0.03, stretcher 0.01, parachute 0.01, 2.8ms
Speed: 0.3ms pre-process, 2.8ms inference, 5.5ms NMS per image at shape (1, 3, 224, 224)
Results saved to runs/predict-cls/exp22


In [18]:
# Helicoptero
import requests
image_url = "https://i.imgur.com/rvFF7jA.jpg"
response = requests.get(image_url)
response.raise_for_status()
with open('helicoptero.jpg', 'wb') as handler:
    handler.write(response.content)

In [46]:
!python classify/predict.py --weights ./weights/yolov5x-cls.pt --source helicoptero.jpg

classify/predict: weights=['./weights/yolov5n-cls.pt'], source=helicoptero.jpg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-230-g53efd07 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 117 layers, 2489464 parameters, 0 gradients, 3.9 GFLOPs
image 1/1 /content/yolov5/helicoptero.jpg: 224x224 stretcher 0.13, lifeboat 0.06, military aircraft 0.06, ski 0.04, container ship 0.04, 2.9ms
Speed: 0.3ms pre-process, 2.9ms inference, 5.3ms NMS per image at shape (1, 3, 224, 224)
Results saved to runs/predict-cls/exp27


In [20]:
# Moto
import requests
image_url = "https://i.imgur.com/ASwjAT5.jpg"
response = requests.get(image_url)
response.raise_for_status()
with open('moto.jpg', 'wb') as handler:
    handler.write(response.content)

In [48]:
!python classify/predict.py --weights ./weights/yolov5m-cls.pt --source moto.jpg

classify/predict: weights=['./weights/yolov5m-cls.pt'], source=moto.jpg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-230-g53efd07 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 166 layers, 12947192 parameters, 0 gradients, 31.7 GFLOPs
image 1/1 /content/yolov5/moto.jpg: 224x224 moped 0.64, scooter 0.17, disc brake 0.06, crash helmet 0.05, snowmobile 0.01, 5.2ms
Speed: 0.3ms pre-process, 5.2ms inference, 5.9ms NMS per image at shape (1, 3, 224, 224)
Results saved to runs/predict-cls/exp29


In [22]:
# Barco
import requests
image_url = "https://i.imgur.com/tUbcIGU.jpg"
response = requests.get(image_url)
response.raise_for_status()
with open('barco.jpg', 'wb') as handler:
    handler.write(response.content)

In [44]:
!python classify/predict.py --weights ./weights/yolov5l-cls.pt --source barco.jpg

classify/predict: weights=['./weights/yolov5l-cls.pt'], source=barco.jpg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-230-g53efd07 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 215 layers, 26535976 parameters, 0 gradients, 69.3 GFLOPs
image 1/1 /content/yolov5/barco.jpg: 224x224 lakeshore 0.25, seashore 0.14, shipwreck 0.12, motorboat 0.09, lifeboat 0.08, 8.4ms
Speed: 0.3ms pre-process, 8.4ms inference, 5.7ms NMS per image at shape (1, 3, 224, 224)
Results saved to runs/predict-cls/exp25


In [24]:
#carro
import requests
image_url = "https://i.imgur.com/GB9Tihf.jpg"
response = requests.get(image_url)
response.raise_for_status()
with open('carro.jpg', 'wb') as handler:
    handler.write(response.content)

In [50]:
!python classify/predict.py --weights ./weights/yolov5x-cls.pt --source carro.jpg

classify/predict: weights=['./weights/yolov5x-cls.pt'], source=carro.jpg, data=data/coco128.yaml, imgsz=[224, 224], device=, view_img=False, save_txt=False, nosave=False, augment=False, visualize=False, update=False, project=runs/predict-cls, name=exp, exist_ok=False, half=False, dnn=False, vid_stride=1
YOLOv5 🚀 v7.0-230-g53efd07 Python-3.10.12 torch-2.1.0+cu118 CUDA:0 (Tesla T4, 15102MiB)

Fusing layers... 
Model summary: 264 layers, 48072600 parameters, 0 gradients, 129.9 GFLOPs
image 1/1 /content/yolov5/carro.jpg: 224x224 sports car 0.95, race car 0.02, convertible 0.01, car wheel 0.00, grille 0.00, 12.9ms
Speed: 0.3ms pre-process, 12.9ms inference, 5.8ms NMS per image at shape (1, 3, 224, 224)
Results saved to runs/predict-cls/exp31
